### 1) Importing all the necessary libraries

In [ ]:
import ase 
from ase.io import read, write
from ase.io import read, write
from ase.calculators.emt import EMT               # replace with your preferred calculator
from ase.optimize import BFGS
import numpy as np

### 2) Read data and rescale configuration box as needed

In [ ]:
# --- 1) Read & compress as before ---
atoms = read('WaterBox.data', format='lammps-data')
atoms.set_pbc(True)
# (set cell manually if your .xyz lacks one)
# atoms.set_cell([Lx, Ly, Lz])

# compute current density
mass_g     = atoms.get_masses().sum() * 1.660539e-24  
vol_cm3    = atoms.get_volume()   * 1e-24          
rho_current = mass_g / vol_cm3
rho_target  = 1.08
f = (rho_current / rho_target)**(1/3)

print(rho_current, rho_target, f)

# scale both cell & positions
atoms.set_cell(atoms.get_cell() * f, scale_atoms=True)
atoms.center()

print(atoms.get_chemical_formula())

# 2) Replace all “X” symbols with “H”
symbols = atoms.get_chemical_symbols()
symbols = ['H' if s == 'X' else s for s in symbols]
atoms.set_chemical_symbols(symbols)
print(atoms.get_chemical_formula())



0.9653266722483252 1.08 0.9632747657416558
O109X218
H218O109


### 3) Import the PET-MAD environment to relax the structure

In [17]:
from pet_mad.calculator import PETMADCalculator

pet_mad_calculator = PETMADCalculator(version="latest", device="cpu")
atoms.calc = pet_mad_calculator
opt = BFGS(atoms, logfile='pos_opt.log')
opt.run(fmax=0.2)

np.True_

### 4) Save the new relaxed starting configuration 

In [18]:
# 6) Write out the final relaxed structure
write('water_relaxed.data', atoms, format='lammps-data')